# **Extract DayMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: April 4, 2022

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python.

In [12]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

In [13]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

In [14]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

[Errno 2] No such file or directory: 'drive/MyDrive/spatial_colab/datasets/'
/content/drive/MyDrive/spatial_colab/datasets
2021_clip/      climate_stats/         IrrMapper/    subset_test_shp/
Boise_CDL/      diversion_timeseries/  LBRB_shp/
CDL_reproject/  irrigation_companies/  lcmap_files/
CDL_subsets/    irrig_lbrb/            masked/


In [16]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = 'subset_test_shp/subset_class/subset_class.shp'
subset = emap.shp_to_ee(shp_file)

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [17]:
## --------------------------------------- ##
## 2. IMPORT THE DAYMET DATA FOR MAX TEMPS ##
## --------------------------------------- ##

years = np.arange(1985,2021)
mean_max = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection
  mxtmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean() #select the bands to analyze
  mean_max.append(mxtmp) #calculate the mean across all pixels

means_max_temp = ee.ImageCollection(mean_max) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [18]:
## ------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ------------------------------------------- ##

years = np.arange(1985,2021)
sum_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-01-01'), (str(years[i])+'-12-31')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum() #select the bands to analyze
  sum_pr.append(prcp) #calculate the mean across all pixels

sum_precip = ee.ImageCollection(sum_pr) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [19]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/maxtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/precip_stats.csv')
emap.zonal_statistics(sum_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/maxtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/precip_stats.csv


In [20]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

precip = pd.read_csv('climate_stats/precip_stats.csv')
max_temp = pd.read_csv('climate_stats/maxtemp_stats.csv')

names = precip['DIV_NAME_']
print(names)

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['Precip_mm'] = precip.iloc[0,0:36].values
  df['Max_temp'] = max_temp.iloc[0,0:36].values
  out_path = os.path.join('climate_stats/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


0                                     PioneerIrrigation
1                                      Ballentyne Ditch
2                   Eureka #2_Upper Center Point Feeder
3                     Farmers Cooperative Seebree Canal
4                                   Farmers Union Ditch
5     Middleton Irrigation Association and Middleton...
6                     New Dry Creek Canal_PD from River
7                                       Riverside Canal
8                                            Rossi Mill
9                    Settlers Irrigation District Ditch
10                                     Siebenberg Canal
Name: DIV_NAME_, dtype: object


In [21]:
## -------------------------- ## 
## 2. Import the GridMET data ##
## -------------------------- ## 

dataset = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('1987-01-01', '2020-12-31')
dataset.first()
max_temp = dataset.select('tmmx') \
            .map(lambda image:image.clip(subset))

max_tempVis = {
    'min': 290.0,
    'max': 314.0,
    'palette': ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c']

}

map = emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(max_temp , max_tempVis,"Maximum Temperature")

map.addLayerControl()
map
